# Getting Started with Amazon SageMaker Automatic Model Tuning

## Background

Machine learning model training is controlled by the set of values that is refered to as hyperparameters. In contrast to parameters plugged into optimization functions, such as node weights or bias, hyperparameters are defined before model training. One can tune them manually to reach to better model performance by changing the values based on one's expertise. Alternatively, one can run hyperparameter optimization and tune these parameters automatically. 

Amazon SageMaker Automatic Model Tuning reduces the undifferentiated heavy lifting of researching the hyperparameter space, by launching training jobs with several sets of hyperparameter combinations and provides the set of best performing values as a result.

This tutorial will walk you through SageMaker Automatic Model Tuning (AMT) using a built-in XGBoost algorithm on Amazon SageMaker. Additional information can be found on documentation pages:
* For running a simple hyperparameter tuning job: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex.html
* For HyperParameterTuner API: https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html

## Overview

The notebook is split into the following sections:
* Setup and Imports
* Load and Prepare dataset
* Train a SageMaker Built-In XGBoost Algorithm
* Train and Tune a SageMaker Built-In XGBoost Algorithm
* View the AMT job statistics 
* Visualize AMT job results and tuned Hyperparameters


### Setup and Imports

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
#import sys
import sagemaker

#!{sys.executable} -m pip install --upgrade pip       --quiet # upgrade pip to the latest vesion
#!{sys.executable} -m pip install --upgrade sagemaker --quiet # upgrade SageMaker to the latest vesion
sagemaker.__version__

'2.112.0'

In [25]:
import io
import os
import argparse
import traceback
import boto3
import numpy as np
import pandas as pd

from sklearn import datasets
from pathlib import Path

In [26]:
# SDK setup
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm = boto3.client('sagemaker')
boto_sess = boto3.Session(region_name=region)
sm_sess = sagemaker.session.Session(boto_session=boto_sess, sagemaker_client=sm)

In [27]:
BUCKET = sm_sess.default_bucket()
PREFIX = 'amt-visualize-demo/data'
s3_data_url = f's3://{BUCKET}/{PREFIX}'

# Eventual output destination for our XGBoost model
output_path = f's3://{BUCKET}/{PREFIX}/output'
print(output_path)

s3://sagemaker-us-east-1-632581975302/amt-visualize-demo/data/output


## Load and Prepare dataset 

In [28]:
!mkdir -p data

The dataset used in the notebook is a scikit-learn library copy of the test set of the UCI ML hand-written digits datasets https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits. Each datapoint is a 8x8 image of a digit. 

In [34]:
from sklearn import datasets

digits         = datasets.load_digits()
digits_df      = pd.DataFrame(digits.data)
digits_df['y'] = digits.target
digits_df.insert(0, 'y', digits_df.pop('y')) # XGBoost expects the target to be the first column 

In [35]:
# Randomly sort the data then split out into train 70% and validation 30%
train_data, valid_data= np.split(
    digits_df, [int(0.7 * len(digits_df))]
)

In [36]:
train_data.to_csv("data/train.csv", index=False, header=False)
valid_data.to_csv("data/valid.csv", index=False, header=False) # valid

We upload train and validation datasets into Amazon S3. Amazon SageMaker will interact with the data directly from S3.

In [37]:
boto_sess.resource("s3").Bucket(BUCKET).Object(os.path.join(PREFIX, "train/train.csv")
                                                 ).upload_file("data/train.csv")
boto_sess.resource("s3").Bucket(BUCKET).Object(os.path.join(PREFIX, "valid/valid.csv")
                                                 ).upload_file("data/valid.csv")

We will use the built-in algorithm that comes in an image URI as described in the docs here:
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

## Train an Amazon SageMaker Built-In XGBoost Algorithm

In [39]:
%%time
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

hyperparameters = {
        "num_class": "10",
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"1",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"multi:softmax",
        "eval_metric":"accuracy",
        "num_round":"50"}

# lookup the XGBoost image URI and builds an XGBoost container
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")
display(xgboost_container)

# construct a SageMaker estimator that calls the XGBoost container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=role,
                                          instance_count=1, 
                                          instance_type='ml.m5.large', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

# define the data type and paths to the training and validation datasets
s3_input_train = TrainingInput(
    s3_data=f's3://{BUCKET}/{PREFIX}/train', content_type="csv")
s3_input_valid = TrainingInput(
    s3_data=f's3://{BUCKET}/{PREFIX}/valid', content_type="csv")

# execute the XGBoost training job
estimator.fit({'train': s3_input_train, 'validation': s3_input_valid})

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1'

2022-10-14 13:24:38 Starting - Starting the training job...
2022-10-14 13:25:02 Starting - Preparing the instances for trainingProfilerReport-1665753877: InProgress
.........
2022-10-14 13:26:29 Downloading - Downloading input data......
2022-10-14 13:27:30 Training - Downloading the training image.....[2022-10-14 13:28:15.093 ip-10-2-111-66.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-14:13:28:15:INFO] Imported framework sagemaker_xgboost_container.training
[2022-10-14:13:28:15:INFO] Failed to parse hyperparameter eval_metric value accuracy to Json.
Returning the value itself
[2022-10-14:13:28:15:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2022-10-14:13:28:15:INFO] No GPUs detected (normal if no gpus installed)
[2022-10-14:13:28:15:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-10-14:13:28:15:INFO] Determined delimiter of CSV input is ','
[2022-10-14:13:28:15:INFO] Determined delimite

## Train and Tune an Amazon SageMaker Built-In XGBoost Algorithm

Amazon SageMaker AMT now orchestrates different trials. We use `tuner.wait()` to pause notebook execution until the AMT job is completed. Depending on the number of jobs and teh configuration of their paralelization this may take a while. For the example above it may take around 10 minutes. During this time you can view the status of your jobs in the console by navigating to Amazon SageMaker > Training > Hyperparameter tuning jobs.

For more information on AMT job monitoring, see: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html

In [40]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

n_jobs = 50
n_parallel_jobs = 2

# redundant declaration - included for visibility 
hyperparameters = {
        "num_class": "10",
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"1",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"multi:softmax",
        "eval_metric":"accuracy",
        "num_round":"50"}

hpt_ranges = {'eta': IntegerParameter(0, 1),
              'alpha': IntegerParameter(0, 2),
              'min_child_weight': IntegerParameter(1, 10),
              'max_depth': IntegerParameter(1, 20)
             }

tuner_parameters = {'estimator': estimator,
                    'base_tuning_job_name': 'bayesian',                   
                    'objective_metric_name': 'validation:accuracy',
                    'objective_type': 'Maximize',
                    'hyperparameter_ranges': hpt_ranges,
                    'strategy': 'Bayesian',
                    'max_jobs': n_jobs,
                    'max_parallel_jobs': n_parallel_jobs}

In [41]:
tuner = HyperparameterTuner(**tuner_parameters)
tuner.fit({'train': s3_input_train, 'validation': s3_input_valid}, wait=False)
tuner_name = tuner.describe()["HyperParameterTuningJobName"]
print(f'tuning job submitted: {tuner_name}.')

tuning job submitted: bayesian-221014-1329.


In [ ]:
tuner.wait()

.............................................................................................................................................................................................................

## View the AMT job statistics and results 

Tuning jobs you have run can be accessed from the Amazon SageMaker console at https://console.aws.amazon.com/sagemaker/. Select Hyperparameter tuning job from the Training menu to see the list. More information here: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html

We can check the results of the HPO jobs and investigate the hyperparameters used, the final value achieved in the objective function and the total training time per job.

#### This can be done either via the Amazon SageMaker Python SDK

In [ ]:
sagemaker.HyperparameterTuningJobAnalytics(tuner_name).dataframe()[:10]

#### Or via the AWS SDK for Python (Boto3)

With the boto3 client we review the results of HPO job using describe_hyper_parameter_tuning_job() function.

In [ ]:
#sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_name)   # to review all statistics
sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_name)['BestTrainingJob']

We can also utilize the boto3 list_training_jobs_for_hyper_parameter_tuning_job() function to review the results sorted by the value of the objective function and including metric definitions. More functions available for Amazon SageMaker with boto3 are described on this page: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html

In [ ]:
hpo_jobs = sm.list_training_jobs_for_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_name,
    MaxResults=100,
    SortBy='FinalObjectiveMetricValue',
    SortOrder='Descending')

for job in hpo_jobs['TrainingJobSummaries']:
    job_descr = sm.describe_training_job(TrainingJobName=job['TrainingJobName'])
    metrics = {m['MetricName']:  m['Value'] for m in job_descr['FinalMetricDataList']}
    print(f'{(job['TrainingJobName']} Metrics: {metrics}')

## Visualize AMT job results and tuned Hyperparameters

Finally, we want to visualise the behaviour of our hyperparameters at different values.

To do this, we are using the altair library, and have written two custom analysis scripts `job_analytics.py` and `reporting_util.py` that we make available with this notebook

In [ ]:
!pip install -Uq altair

In [ ]:
from job_analytics import *
from reporting_util import *

import altair as alt


_ = alt.data_transformers.disable_max_rows()
alt.renderers.enable('mimetype')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)  # Don't truncate TrainingJobName        

Please ensure that the role used by SageMaker allows the cloudwatch:ListMetrics action on IAM.

In [ ]:
analyze_hpo_job(tuner)